In [1]:
import pandas as pd
import json

with open("../compilation-analysis/option_columns.json","r") as f:
    option_columns = json.load(f)

#Find the dataset
df = pd.read_csv("../tuxml-datasets/dataset_encoded_size.csv", dtype={k:"int8" for k in option_columns})

df.query("cid >= 30000", inplace=True)
df.fillna(-1, inplace=True)
df.query("kernel_size >= 0", inplace=True)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import ensemble

def runML_RF(hyperparams, with_tree=False):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="cid").drop(columns="kernel_size"), df["kernel_size"], test_size=0.1)  
    reg = ensemble.RandomForestClassifier(**hyperparams)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    if with_tree:
        return dfErrors["% error"].describe(), reg
    return dfErrors["% error"].describe()


In [7]:

hyperparams = {
    "max_depth":18,
    "n_estimators":50,
    
}
res = runML_RF(hyperparams)
res

count    9248.000000
mean       77.800149
std       166.695037
min         0.015446
25%        16.505824
50%        37.479969
75%        71.754307
max      3451.241541
Name: % error, dtype: float64